In [2]:
import os
import torch
import argparse
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [6]:
# Configurations
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
opt = argparse.Namespace()
opt.latent_size = 64
opt.hidden_size = 256
opt.image_size = 784
opt.num_epochs = 200
opt.batch_size = 100
opt.sample_dir = 'samples'
os.makedirs(opt.sample_dir, exist_ok=True)

In [7]:
# Load dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
mnist = torchvision.datasets.MNIST('./mnist', train=True, transform=transform, download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist, batch_size=opt.batch_size, shuffle=True)

In [11]:
# Generator and discriminator
D = nn.Sequential(
    nn.Linear(opt.image_size, opt.hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(opt.hidden_size, opt.hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(opt.hidden_size, 1),
    nn.Sigmoid(),
)

G = nn.Sequential(
    nn.Linear(opt.latent_size, opt.hidden_size),
    nn.ReLU(),
    nn.Linear(opt.hidden_size, opt.hidden_size),
    nn.ReLU(),
    nn.Linear(opt.hidden_size, opt.image_size),
    nn.Tanh(),
)

D = D.to(device)
G = G.to(device)
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)


def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [14]:
# Training
total_step = len(data_loader)
for epoch in range(opt.num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(opt.batch_size, -1).to(device)
        real_labels = torch.ones(opt.batch_size, 1).to(device)
        fake_labels = torch.zeros(opt.batch_size, 1).to(device)
        
        ######## Train the discriminator ########
        # Compute BCE loss using real images
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCE loss using fake images
        z = torch.randn(opt.batch_size, opt.latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Back propagation
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        ######## Train the generator ########
        # Compute BCE loss using fake images
        z = torch.randn(opt.batch_size, opt.latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # Back propagation
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' .format(epoch, opt.num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(opt.sample_dir, 'real_images.png'))
        
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(opt.sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0694, g_loss: 5.3000, D(x): 0.99, D(G(z)): 0.06
Epoch [0/200], Step [400/600], d_loss: 0.0380, g_loss: 5.5166, D(x): 0.99, D(G(z)): 0.03
Epoch [0/200], Step [600/600], d_loss: 0.1086, g_loss: 4.4043, D(x): 0.97, D(G(z)): 0.07
Epoch [1/200], Step [200/600], d_loss: 0.5547, g_loss: 3.5855, D(x): 0.93, D(G(z)): 0.31
Epoch [1/200], Step [400/600], d_loss: 0.1471, g_loss: 4.3318, D(x): 0.96, D(G(z)): 0.08
Epoch [1/200], Step [600/600], d_loss: 0.2768, g_loss: 3.4308, D(x): 0.90, D(G(z)): 0.11
Epoch [2/200], Step [200/600], d_loss: 0.4917, g_loss: 3.2210, D(x): 0.81, D(G(z)): 0.13
Epoch [2/200], Step [400/600], d_loss: 1.2040, g_loss: 1.6527, D(x): 0.68, D(G(z)): 0.38
Epoch [2/200], Step [600/600], d_loss: 0.3248, g_loss: 4.6024, D(x): 0.88, D(G(z)): 0.06
Epoch [3/200], Step [200/600], d_loss: 0.4566, g_loss: 2.8978, D(x): 0.86, D(G(z)): 0.17
Epoch [3/200], Step [400/600], d_loss: 0.2951, g_loss: 4.1879, D(x): 0.91, D(G(z)): 0.10
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2213, g_loss: 3.5913, D(x): 0.93, D(G(z)): 0.09
Epoch [31/200], Step [200/600], d_loss: 0.2688, g_loss: 3.5223, D(x): 0.88, D(G(z)): 0.04
Epoch [31/200], Step [400/600], d_loss: 0.2710, g_loss: 3.8114, D(x): 0.90, D(G(z)): 0.07
Epoch [31/200], Step [600/600], d_loss: 0.1801, g_loss: 4.0652, D(x): 0.95, D(G(z)): 0.07
Epoch [32/200], Step [200/600], d_loss: 0.3518, g_loss: 3.0262, D(x): 0.86, D(G(z)): 0.08
Epoch [32/200], Step [400/600], d_loss: 0.1493, g_loss: 4.4532, D(x): 0.94, D(G(z)): 0.05
Epoch [32/200], Step [600/600], d_loss: 0.5605, g_loss: 4.1430, D(x): 0.93, D(G(z)): 0.23
Epoch [33/200], Step [200/600], d_loss: 0.2223, g_loss: 3.7490, D(x): 0.93, D(G(z)): 0.09
Epoch [33/200], Step [400/600], d_loss: 0.3975, g_loss: 3.5904, D(x): 0.90, D(G(z)): 0.14
Epoch [33/200], Step [600/600], d_loss: 0.2764, g_loss: 5.3074, D(x): 0.98, D(G(z)): 0.16
Epoch [34/200], Step [200/600], d_loss: 0.3221, g_loss: 4.1065, D(x): 0.88, D(G(z)): 0.04
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.8525, g_loss: 2.3790, D(x): 0.69, D(G(z)): 0.21
Epoch [61/200], Step [600/600], d_loss: 0.6972, g_loss: 1.5434, D(x): 0.81, D(G(z)): 0.29
Epoch [62/200], Step [200/600], d_loss: 0.5673, g_loss: 2.2204, D(x): 0.79, D(G(z)): 0.17
Epoch [62/200], Step [400/600], d_loss: 0.7993, g_loss: 2.0573, D(x): 0.70, D(G(z)): 0.19
Epoch [62/200], Step [600/600], d_loss: 0.6367, g_loss: 1.7948, D(x): 0.77, D(G(z)): 0.20
Epoch [63/200], Step [200/600], d_loss: 0.7693, g_loss: 2.5957, D(x): 0.69, D(G(z)): 0.18
Epoch [63/200], Step [400/600], d_loss: 0.9364, g_loss: 1.8150, D(x): 0.80, D(G(z)): 0.35
Epoch [63/200], Step [600/600], d_loss: 0.5047, g_loss: 2.9084, D(x): 0.81, D(G(z)): 0.14
Epoch [64/200], Step [200/600], d_loss: 0.4712, g_loss: 2.2551, D(x): 0.84, D(G(z)): 0.19
Epoch [64/200], Step [400/600], d_loss: 0.6667, g_loss: 2.1150, D(x): 0.77, D(G(z)): 0.20
Epoch [64/200], Step [600/600], d_loss: 0.7653, g_loss: 1.8559, D(x): 0.90, D(G(z)): 0.37
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.8488, g_loss: 1.8191, D(x): 0.79, D(G(z)): 0.32
Epoch [92/200], Step [400/600], d_loss: 0.8067, g_loss: 2.0722, D(x): 0.76, D(G(z)): 0.26
Epoch [92/200], Step [600/600], d_loss: 0.7020, g_loss: 2.4495, D(x): 0.72, D(G(z)): 0.17
Epoch [93/200], Step [200/600], d_loss: 0.7865, g_loss: 2.3457, D(x): 0.74, D(G(z)): 0.25
Epoch [93/200], Step [400/600], d_loss: 0.8434, g_loss: 2.1071, D(x): 0.70, D(G(z)): 0.24
Epoch [93/200], Step [600/600], d_loss: 0.7346, g_loss: 1.9919, D(x): 0.73, D(G(z)): 0.22
Epoch [94/200], Step [200/600], d_loss: 0.8794, g_loss: 1.7787, D(x): 0.77, D(G(z)): 0.33
Epoch [94/200], Step [400/600], d_loss: 0.6737, g_loss: 2.1434, D(x): 0.72, D(G(z)): 0.19
Epoch [94/200], Step [600/600], d_loss: 0.6739, g_loss: 1.8190, D(x): 0.73, D(G(z)): 0.18
Epoch [95/200], Step [200/600], d_loss: 0.9029, g_loss: 1.6790, D(x): 0.70, D(G(z)): 0.26
Epoch [95/200], Step [400/600], d_loss: 0.9636, g_loss: 2.0211, D(x): 0.70, D(G(z)): 0.27
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8291, g_loss: 1.8477, D(x): 0.76, D(G(z)): 0.31
Epoch [122/200], Step [600/600], d_loss: 0.6349, g_loss: 2.7256, D(x): 0.81, D(G(z)): 0.22
Epoch [123/200], Step [200/600], d_loss: 0.8441, g_loss: 1.3889, D(x): 0.72, D(G(z)): 0.30
Epoch [123/200], Step [400/600], d_loss: 0.9717, g_loss: 1.7682, D(x): 0.61, D(G(z)): 0.22
Epoch [123/200], Step [600/600], d_loss: 0.5595, g_loss: 2.1728, D(x): 0.76, D(G(z)): 0.18
Epoch [124/200], Step [200/600], d_loss: 0.5473, g_loss: 2.0302, D(x): 0.80, D(G(z)): 0.22
Epoch [124/200], Step [400/600], d_loss: 0.7158, g_loss: 1.5375, D(x): 0.80, D(G(z)): 0.31
Epoch [124/200], Step [600/600], d_loss: 0.7817, g_loss: 1.8656, D(x): 0.71, D(G(z)): 0.25
Epoch [125/200], Step [200/600], d_loss: 0.8413, g_loss: 2.1624, D(x): 0.67, D(G(z)): 0.14
Epoch [125/200], Step [400/600], d_loss: 0.8320, g_loss: 2.1819, D(x): 0.72, D(G(z)): 0.25
Epoch [125/200], Step [600/600], d_loss: 0.8693, g_loss: 1.6756, D(x): 0.73, D(G(z)): 0.32

Epoch [152/200], Step [600/600], d_loss: 0.7322, g_loss: 2.0978, D(x): 0.74, D(G(z)): 0.23
Epoch [153/200], Step [200/600], d_loss: 0.8398, g_loss: 1.7038, D(x): 0.68, D(G(z)): 0.22
Epoch [153/200], Step [400/600], d_loss: 0.8251, g_loss: 1.6193, D(x): 0.68, D(G(z)): 0.23
Epoch [153/200], Step [600/600], d_loss: 0.9815, g_loss: 1.5129, D(x): 0.65, D(G(z)): 0.28
Epoch [154/200], Step [200/600], d_loss: 1.0123, g_loss: 1.5456, D(x): 0.70, D(G(z)): 0.35
Epoch [154/200], Step [400/600], d_loss: 0.7784, g_loss: 1.4133, D(x): 0.78, D(G(z)): 0.31
Epoch [154/200], Step [600/600], d_loss: 0.9975, g_loss: 1.6651, D(x): 0.60, D(G(z)): 0.21
Epoch [155/200], Step [200/600], d_loss: 0.9399, g_loss: 1.6671, D(x): 0.67, D(G(z)): 0.29
Epoch [155/200], Step [400/600], d_loss: 1.0999, g_loss: 1.3081, D(x): 0.68, D(G(z)): 0.33
Epoch [155/200], Step [600/600], d_loss: 0.8857, g_loss: 1.5576, D(x): 0.68, D(G(z)): 0.25
Epoch [156/200], Step [200/600], d_loss: 0.9966, g_loss: 1.6261, D(x): 0.68, D(G(z)): 0.32

Epoch [183/200], Step [200/600], d_loss: 0.8972, g_loss: 1.5209, D(x): 0.68, D(G(z)): 0.26
Epoch [183/200], Step [400/600], d_loss: 0.9445, g_loss: 1.2271, D(x): 0.76, D(G(z)): 0.38
Epoch [183/200], Step [600/600], d_loss: 0.9372, g_loss: 1.3869, D(x): 0.74, D(G(z)): 0.37
Epoch [184/200], Step [200/600], d_loss: 1.1443, g_loss: 1.4700, D(x): 0.59, D(G(z)): 0.28
Epoch [184/200], Step [400/600], d_loss: 0.9309, g_loss: 1.4704, D(x): 0.71, D(G(z)): 0.33
Epoch [184/200], Step [600/600], d_loss: 0.9550, g_loss: 1.5937, D(x): 0.66, D(G(z)): 0.25
Epoch [185/200], Step [200/600], d_loss: 0.7873, g_loss: 1.5220, D(x): 0.79, D(G(z)): 0.31
Epoch [185/200], Step [400/600], d_loss: 0.9314, g_loss: 1.8968, D(x): 0.62, D(G(z)): 0.22
Epoch [185/200], Step [600/600], d_loss: 0.9100, g_loss: 1.2935, D(x): 0.70, D(G(z)): 0.32
Epoch [186/200], Step [200/600], d_loss: 0.8735, g_loss: 1.2872, D(x): 0.76, D(G(z)): 0.34
Epoch [186/200], Step [400/600], d_loss: 0.8554, g_loss: 1.7999, D(x): 0.68, D(G(z)): 0.25